In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import csv
import requests
import os
from zipfile import ZipFile
from datetime import date, datetime

In [2]:
yearDate = date.today().strftime('%m%d%y')
print(str(datetime.now()) + " Running 5_COVID_Hospital_ICU_Process")
print(yearDate)

2020-11-15 12:48:19.852204 Running 5_COVID_Hospital_ICU_Process
111520


In [3]:
dataFile_icu = "data/ICU/ICU_BedsCounty1_data{}.csv".format(yearDate)
dataFile_newhosp = "data/ICU/COVID_Hospitalizations_County1_data{}.csv".format(yearDate)
dataFile_hp = "data/ICU/Hospital_BedsCounty1_data{}.csv".format(yearDate)

In [4]:
df_ICU = pd.read_csv(dataFile_icu, sep=',',encoding='utf-8')
df_ICU.head()

,County,Measure Names,Measure Values
0,WASHINGTON,Total PediatricICU Capacity,0.0
1,WALTON,Total PediatricICU Capacity,0.0
2,VOLUSIA,Total PediatricICU Capacity,0.0
3,UNION,Total PediatricICU Capacity,0.0
4,TAYLOR,Total PediatricICU Capacity,0.0


In [5]:
df_newhosp = pd.read_csv(dataFile_newhosp,sep=',',encoding='utf-8')
df_newhosp.head()

,County,COVID Hospitalizations
0,WASHINGTON,6
1,WALTON,7
2,VOLUSIA,61
3,UNION,2
4,TAYLOR,0


In [6]:
df_hp = pd.read_csv(dataFile_hp, sep=',',encoding='utf-8')
df_hp.head()

,County,Measure Names,Measure Values
0,WASHINGTON,Total Staffed Bed Capacity,59.0
1,WALTON,Total Staffed Bed Capacity,115.0
2,VOLUSIA,Total Staffed Bed Capacity,1534.0
3,UNION,Total Staffed Bed Capacity,76.0
4,TAYLOR,Total Staffed Bed Capacity,12.0


In [7]:
df_ICU_pivot = pd.pivot_table(df_ICU,values='Measure Values',index= 'County',columns = 'Measure Names',aggfunc=np.sum)
df_ICU_pivot.head()

Measure Names,Adult ICU Census,Available Adult ICU,Available Adult ICU%,Available Pediatric ICU,Available Pediatric ICU%,Pediatric ICU Census,Total AdultICU Capacity,Total PediatricICU Capacity
County,,,,,,,,
ALACHUA,276.0,34.0,0.109677,8.0,0.170213,39.0,310.0,47.0
BAKER,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0
BAY,54.0,0.0,0.000000,4.0,1.000000,0.0,54.0,4.0
BRADFORD,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0
BREVARD,120.0,47.0,0.281437,0.0,0.000000,0.0,167.0,0.0


In [8]:
df_hp_pivot = pd.pivot_table(df_hp,values='Measure Values',index= 'County',columns = 'Measure Names',aggfunc=np.sum)
df_hp_pivot.head()

Measure Names,Available,Available Capacity,Bed Census,Total Staffed Bed Capacity
County,,,,
ALACHUA,237.0,0.136285,1502.0,1739.0
BAKER,116.0,0.185600,509.0,625.0
BAY,96.0,0.176796,447.0,543.0
BRADFORD,5.0,0.500000,5.0,10.0
BREVARD,528.0,0.323728,1103.0,1631.0


In [9]:
df_merge = pd.merge(df_ICU_pivot, df_hp_pivot, left_on=  ['County'],right_on= ['County'], how = 'outer')
df_merge.head()

Measure Names,Adult ICU Census,Available Adult ICU,Available Adult ICU%,Available Pediatric ICU,Available Pediatric ICU%,Pediatric ICU Census,Total AdultICU Capacity,Total PediatricICU Capacity,Available,Available Capacity,Bed Census,Total Staffed Bed Capacity
County,,,,,,,,,,,,
ALACHUA,276.0,34.0,0.109677,8.0,0.170213,39.0,310.0,47.0,237.0,0.136285,1502.0,1739.0
BAKER,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,116.0,0.185600,509.0,625.0
BAY,54.0,0.0,0.000000,4.0,1.000000,0.0,54.0,4.0,96.0,0.176796,447.0,543.0
BRADFORD,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,5.0,0.500000,5.0,10.0
BREVARD,120.0,47.0,0.281437,0.0,0.000000,0.0,167.0,0.0,528.0,0.323728,1103.0,1631.0


In [10]:
df_merge_2 = pd.merge(df_merge, df_newhosp, left_on=  ['County'],right_on= ['County'], how = 'outer')
df_merge_2.head()

,County,Adult ICU Census,Available Adult ICU,Available Adult ICU%,Available Pediatric ICU,Available Pediatric ICU%,Pediatric ICU Census,Total AdultICU Capacity,Total PediatricICU Capacity,Available,Available Capacity,Bed Census,Total Staffed Bed Capacity,COVID Hospitalizations
0,ALACHUA,276.0,34.0,0.109677,8.0,0.170213,39.0,310.0,47.0,237.0,0.136285,1502.0,1739.0,72
1,BAKER,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,116.0,0.185600,509.0,625.0,0
2,BAY,54.0,0.0,0.000000,4.0,1.000000,0.0,54.0,4.0,96.0,0.176796,447.0,543.0,23
3,BRADFORD,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,5.0,0.500000,5.0,10.0,0
4,BREVARD,120.0,47.0,0.281437,0.0,0.000000,0.0,167.0,0.0,528.0,0.323728,1103.0,1631.0,73


In [11]:
df_merge_3 = df_merge_2.append(df_merge_2.sum(numeric_only=True), ignore_index=True)

In [12]:
df_merge_3['County'] = df_merge_3['County'].replace(np.nan, 'A State')
df_merge_3.iat[58,3] = df_merge_3.iloc[58][2]/df_merge_3.iloc[58][7]
df_merge_3.iat[58,5] = df_merge_3.iloc[58][4]/df_merge_3.iloc[58][8]
df_merge_3.iat[58,10] = df_merge_3.iloc[58][9]/df_merge_3.iloc[58][12]

In [13]:
df_merge_3

,County,Adult ICU Census,Available Adult ICU,Available Adult ICU%,Available Pediatric ICU,Available Pediatric ICU%,Pediatric ICU Census,Total AdultICU Capacity,Total PediatricICU Capacity,Available,Available Capacity,Bed Census,Total Staffed Bed Capacity,COVID Hospitalizations
0,ALACHUA,276.0,34.0,0.109677,8.0,0.170213,39.0,310.0,47.0,237.0,0.136285,1502.0,1739.0,72.0
1,BAKER,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,116.0,0.185600,509.0,625.0,0.0
2,BAY,54.0,0.0,0.000000,4.0,1.000000,0.0,54.0,4.0,96.0,0.176796,447.0,543.0,23.0
3,BRADFORD,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,5.0,0.500000,5.0,10.0,0.0
4,BREVARD,120.0,47.0,0.281437,0.0,0.000000,0.0,167.0,0.0,528.0,0.323728,1103.0,1631.0,73.0
5,BROWARD,356.0,106.0,0.229437,36.0,0.444444,45.0,462.0,81.0,1124.0,0.216947,4057.0,5181.0,284.0
6,CALHOUN,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,20.0,20.0,0.0
7,CHARLOTTE,49.0,4.0,0.075472,0.0,0.000000,0.0,53.0,0.0,187.0,0.310116,416.0,603.0,45.0
8,CITRUS,33.0,6.0,0.153846,0.0,0.000000,0.0,39.0,0.0,43.0,0.146259,251.0,294.0,40.0
9,CLAY,39.0,11.0,0.220000,3.0,0.750000,1.0,50.0,4.0,94.0,0.168157,465.0,559.0,28.0


In [14]:
result_folder = 'results/{}'.format(yearDate)
if not os.path.exists(result_folder):
    os.makedirs(result_folder)

In [15]:
df_merge_3.to_excel(os.path.join(result_folder,"FL_County_Hospital_ICU{}.xlsx".format(yearDate)))

In [16]:
df_merge_3['Avail_ICU']=(df_merge_3['Available Adult ICU'] + df_merge_3['Available Pediatric ICU'])
df_merge_3.head()

,County,Adult ICU Census,Available Adult ICU,Available Adult ICU%,Available Pediatric ICU,Available Pediatric ICU%,Pediatric ICU Census,Total AdultICU Capacity,Total PediatricICU Capacity,Available,Available Capacity,Bed Census,Total Staffed Bed Capacity,COVID Hospitalizations,Avail_ICU
0,ALACHUA,276.0,34.0,0.109677,8.0,0.170213,39.0,310.0,47.0,237.0,0.136285,1502.0,1739.0,72.0,42.0
1,BAKER,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,116.0,0.185600,509.0,625.0,0.0,0.0
2,BAY,54.0,0.0,0.000000,4.0,1.000000,0.0,54.0,4.0,96.0,0.176796,447.0,543.0,23.0,4.0
3,BRADFORD,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,5.0,0.500000,5.0,10.0,0.0,0.0
4,BREVARD,120.0,47.0,0.281437,0.0,0.000000,0.0,167.0,0.0,528.0,0.323728,1103.0,1631.0,73.0,47.0


In [17]:
df_rename = df_merge_3.rename(columns={"Adult ICU Census":"AICU_Census", "Available Adult ICU":"AICU_Avail",\
                                      "Available Adult ICU%":"A_ICU", "Pediatric ICU Census":"PICU_Census",\
                                      "Available Pediatric ICU":"PICU_Avail","Available Pediatric ICU%":"P_ICU",\
                                      "Total AdultICU Capacity":"T_AICU", "Total PediatricICU Capacity":"T_PICU",\
                                      "Available":"Bed_Avail", "Available Capacity":"Bed_AP",\
                                      "Total Staffed Bed Capacity":"T_bed", 'COVID Hospitalizations':'New_hosp'})
df_rename.head()

,County,AICU_Census,AICU_Avail,A_ICU,PICU_Avail,P_ICU,PICU_Census,T_AICU,T_PICU,Bed_Avail,Bed_AP,Bed Census,T_bed,New_hosp,Avail_ICU
0,ALACHUA,276.0,34.0,0.109677,8.0,0.170213,39.0,310.0,47.0,237.0,0.136285,1502.0,1739.0,72.0,42.0
1,BAKER,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,116.0,0.185600,509.0,625.0,0.0,0.0
2,BAY,54.0,0.0,0.000000,4.0,1.000000,0.0,54.0,4.0,96.0,0.176796,447.0,543.0,23.0,4.0
3,BRADFORD,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,5.0,0.500000,5.0,10.0,0.0,0.0
4,BREVARD,120.0,47.0,0.281437,0.0,0.000000,0.0,167.0,0.0,528.0,0.323728,1103.0,1631.0,73.0,47.0


In [18]:
df_reset_index = df_rename.reset_index()
df_reset_index['County']= df_reset_index['County'].str.title()
df_reset_index.head()

,index,County,AICU_Census,AICU_Avail,A_ICU,PICU_Avail,P_ICU,PICU_Census,T_AICU,T_PICU,Bed_Avail,Bed_AP,Bed Census,T_bed,New_hosp,Avail_ICU
0,0,Alachua,276.0,34.0,0.109677,8.0,0.170213,39.0,310.0,47.0,237.0,0.136285,1502.0,1739.0,72.0,42.0
1,1,Baker,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,116.0,0.185600,509.0,625.0,0.0,0.0
2,2,Bay,54.0,0.0,0.000000,4.0,1.000000,0.0,54.0,4.0,96.0,0.176796,447.0,543.0,23.0,4.0
3,3,Bradford,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,5.0,0.500000,5.0,10.0,0.0,0.0
4,4,Brevard,120.0,47.0,0.281437,0.0,0.000000,0.0,167.0,0.0,528.0,0.323728,1103.0,1631.0,73.0,47.0


In [19]:
gdf_county_polygon = gpd.read_file("shp/Florida_Counties.shp")
gdf_county_polygon.head()

,OBJECTID,DEPCODE,COUNTY,COUNTYNAME,DATESTAMP,ShapeSTAre,ShapeSTLen,geometry
0,1,21,041,Gilchrist,2000-05-16,9.908353e+09,4.873000e+05,"POLYGON ((-82.65814 29.83028, -82.65803 29.822..."
1,2,54,107,Putnam,2000-05-16,2.305869e+10,7.629677e+05,"POLYGON ((-81.58084 29.83956, -81.57893 29.835..."
2,3,62,123,Taylor,2000-05-16,2.891747e+10,8.772527e+05,"MULTIPOLYGON (((-83.73037 30.30358, -83.70368 ..."
3,4,46,091,Okaloosa,2000-05-16,2.562159e+10,1.087058e+06,"MULTIPOLYGON (((-86.39159 30.64970, -86.39175 ..."
4,5,7,013,Calhoun,2000-05-16,1.604809e+10,6.313440e+05,"POLYGON ((-84.93266 30.60637, -84.93287 30.605..."


In [20]:
a = set(df_reset_index['County'].values)
b = set(gdf_county_polygon['COUNTYNAME'].values)
a.difference(b)

{'A State', 'Miami-Dade'}

In [21]:
b.difference(a)

{'Dade',
 'Dixie',
 'Gilchrist',
 'Glades',
 'Hamilton',
 'Jefferson',
 'Lafayette',
 'Levy',
 'Liberty',
 'Wakulla'}

In [22]:
df_reset_index.loc[df_reset_index['County']=='Miami-Dade','County']='Dade'

In [23]:
a = set(df_reset_index['County'].values)
b = set(gdf_county_polygon['COUNTYNAME'].values)
a.difference(b)

{'A State'}

In [24]:
df_merge_poly = pd.merge(df_reset_index, gdf_county_polygon, left_on = ['County'],right_on = ['COUNTYNAME'], how = 'outer')
df_merge_poly

,index,County,AICU_Census,AICU_Avail,A_ICU,PICU_Avail,P_ICU,PICU_Census,T_AICU,T_PICU,...,New_hosp,Avail_ICU,OBJECTID,DEPCODE,COUNTY,COUNTYNAME,DATESTAMP,ShapeSTAre,ShapeSTLen,geometry
0,0.0,Alachua,276.0,34.0,0.109677,8.0,0.170213,39.0,310.0,47.0,...,72.0,42.0,66.0,1.0,001,Alachua,2000-05-16,2.702324e+10,8.927266e+05,"POLYGON ((-82.41869 29.92298, -82.41825 29.922..."
1,1.0,Baker,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,17.0,2.0,003,Baker,2000-05-16,1.636216e+10,6.027173e+05,"POLYGON ((-82.05012 30.36233, -82.04949 30.316..."
2,2.0,Bay,54.0,0.0,0.000000,4.0,1.000000,0.0,54.0,4.0,...,23.0,4.0,15.0,3.0,005,Bay,2000-05-16,2.084232e+10,1.834701e+06,"MULTIPOLYGON (((-85.38551 30.50018, -85.38650 ..."
3,3.0,Bradford,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,40.0,4.0,007,Bradford,2000-05-16,8.359863e+09,5.483267e+05,"POLYGON ((-82.04939 30.14322, -82.04938 30.143..."
4,4.0,Brevard,120.0,47.0,0.281437,0.0,0.000000,0.0,167.0,0.0,...,73.0,47.0,42.0,5.0,009,Brevard,2000-05-16,2.893695e+10,2.600567e+06,"MULTIPOLYGON (((-80.80666 28.61413, -80.80790 ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,26.0,15.0,029,Dixie,2000-05-16,1.963495e+10,7.718929e+05,"MULTIPOLYGON (((-83.24986 29.82302, -83.23964 ..."
64,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,28.0,38.0,075,Levy,2000-05-16,3.122285e+10,1.113561e+06,"MULTIPOLYGON (((-82.92321 29.59089, -82.87460 ..."
65,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,37.0,34.0,067,Lafayette,2000-05-16,1.526457e+10,6.340132e+05,"POLYGON ((-83.24902 30.26044, -83.24970 30.260..."
66,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,41.0,65.0,129,Wakulla,2000-05-16,1.702251e+10,9.167339e+05,"MULTIPOLYGON (((-84.41197 30.30244, -84.37858 ..."


In [25]:
df_merge_poly1 = df_merge_poly.drop(columns=['OBJECTID','DEPCODE','COUNTY','DATESTAMP','ShapeSTLen','ShapeSTAre'])
df_merge_poly1.head()

,index,County,AICU_Census,AICU_Avail,A_ICU,PICU_Avail,P_ICU,PICU_Census,T_AICU,T_PICU,Bed_Avail,Bed_AP,Bed Census,T_bed,New_hosp,Avail_ICU,COUNTYNAME,geometry
0,0.0,Alachua,276.0,34.0,0.109677,8.0,0.170213,39.0,310.0,47.0,237.0,0.136285,1502.0,1739.0,72.0,42.0,Alachua,"POLYGON ((-82.41869 29.92298, -82.41825 29.922..."
1,1.0,Baker,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,116.0,0.185600,509.0,625.0,0.0,0.0,Baker,"POLYGON ((-82.05012 30.36233, -82.04949 30.316..."
2,2.0,Bay,54.0,0.0,0.000000,4.0,1.000000,0.0,54.0,4.0,96.0,0.176796,447.0,543.0,23.0,4.0,Bay,"MULTIPOLYGON (((-85.38551 30.50018, -85.38650 ..."
3,3.0,Bradford,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,5.0,0.500000,5.0,10.0,0.0,0.0,Bradford,"POLYGON ((-82.04939 30.14322, -82.04938 30.143..."
4,4.0,Brevard,120.0,47.0,0.281437,0.0,0.000000,0.0,167.0,0.0,528.0,0.323728,1103.0,1631.0,73.0,47.0,Brevard,"MULTIPOLYGON (((-80.80666 28.61413, -80.80790 ..."


In [27]:
shpfile_name = "Florida_County_Hospital_ICU{}".format(yearDate)
print("Saving {}.shp in results/".format(shpfile_name))
gdf_merge = gpd.GeoDataFrame(df_merge_poly1, geometry='geometry',crs="EPSG:4326")
gdf_merge.to_file(os.path.join(result_folder,"{}.shp".format(shpfile_name)))

Saving Florida_County_Hospital_ICU111520.shp in results/


In [29]:
print("Saving {}.zip in publish/".format(shpfile_name))
with ZipFile(os.path.join('publish', '{}.zip'.format(shpfile_name)), 'w') as zipObj:
    for file in os.listdir(result_folder):
        if shpfile_name in file:
            file_path = os.path.join(result_folder, file)
            print(file)
            try:
                zipObj.write(file_path, os.path.basename(file_path))
            except OSError as err:
                print("OS error: {0}".format(err))  

Saving Florida_County_Hospital_ICU111520.zip in publish/
Florida_County_Hospital_ICU111520.prj
Florida_County_Hospital_ICU111520.dbf
Florida_County_Hospital_ICU111520.shx
Florida_County_Hospital_ICU111520.shp
Florida_County_Hospital_ICU111520.cpg


In [30]:
print(str(datetime.now()) + " Done with 5_COVID_Hospital_ICU_Process")

2020-11-15 12:48:40.398785 Done with 5_COVID_Hospital_ICU_Process
